<a href="https://colab.research.google.com/github/innosoft21/workspace/blob/main/tensorflow_R/Kaggle_Walmart_Store_Sales_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kaggle : Walmart Sales forecasting

In [ ]:
library(tidymodels)
library(tidyverse)
library(magrittr)
library(skimr)
library(knitr)

In [ ]:
file_path <- "../input/walmart-recruiting-store-sales-forecasting/"
files <- list.files(file_path)
files
train <- read_csv(file.path(file_path, "train.csv.zip")) %>% 
  janitor::clean_names()
test <- read_csv(file.path(file_path, "test.csv.zip")) %>% 
  janitor::clean_names()
features <- read_csv(file.path(file_path, "features.csv.zip")) %>% 
  janitor::clean_names()
stores <- read_csv(file.path(file_path, "stores.csv")) %>% 
  janitor::clean_names()

In [ ]:
theme_set(theme_bw())
# size of data
dim(train)
dim(test)
names(train)
train %>% head()
# test
names(test)
test %>% head()
# alldata combine
all_data <- bind_rows(train, test)
names(all_data)
all_data %>% head()
all_data %>% skim()
walmart_recipe <- 
    recipe(weekly_sales ~ .,
           data = all_data) %>% 
    step_mutate(year = lubridate::year(date),
                month = lubridate::month(date)) %>%
    step_rm(date) %>% 
    step_mutate(year = factor(year),
                month = factor(month),
                store = factor(store),
                dept = factor(dept)) %>% 
    step_dummy(all_nominal()) %>%
    step_normalize(all_numeric(), 
                   -all_outcomes()) %>% 
    prep(training = all_data)
walmart_recipe

In [ ]:
all_data2 <- juice(walmart_recipe)
names(all_data2)
head(all_data2)
train_index <- seq_len(nrow(train))
train2 <- all_data2[train_index,]
test2 <- all_data2[-train_index,]
train2 %>% dim()
lm_model <- 
    linear_reg() %>% 
    set_engine("lm")

In [ ]:
lm_form_fit <- 
    lm_model %>% 
    fit(weekly_sales ~ ., data = train2)


In [ ]:
lm_form_fit
result <- predict(lm_form_fit, 
                  new_data = test2)
submission <- read_csv(file.path(file_path, "sampleSubmission.csv.zip"))
submission$Weekly_Sales <- result$.pred
write.csv(submission, row.names = FALSE,
          "baseline-lm-dummy-04022021.csv")